In [1]:
from tqdm import tqdm, tnrange, tqdm_notebook
import json
import pandas as pd
import os
from collections import namedtuple

import utils
import parsers
from nltk.corpus import wordnet as wn
import copy

In [2]:
fine_senses_vocab_path = '../resources/semcor.vocab.WordNet.json'
input_vocab_path = '../resources/semcor.input.vocab.json'
input_antivocab_path = '../resources/semcor.leftout.vocab.json'

In [3]:
senses = utils.json_vocab_reader(fine_senses_vocab_path)
inputs, antivocab = utils.json_vocab_reader(input_vocab_path, input_antivocab_path)

In [4]:
output_vocab = utils.vocab_merge(senses, inputs)
reverse_output_vocab =  dict((v, k) for k, v in output_vocab.items())

In [5]:
import keras

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [6]:
def apply_padding(output, key, maxlen=50, value=1):
    """
    Applies padding to output sequences
    
    param output: dict
    param key: key of dict
    param maxlen: length to pad 
    param value: pad with this value 
    return padded list of lists
    """
    x = output[key]
    if key == 'candidates':
        for candidate in range(len(x)):
            x[candidate] =  x[candidate] + [[value]] * (maxlen-len(x[candidate]))
        return x
    else:
        return list(keras.preprocessing.sequence.pad_sequences(x, truncating='pre', padding='post', maxlen=maxlen, value = value ))


def prepare_sentence_batch(batch_size, training_file_path, antivocab, output_vocab, gold_file_path = None):
    """
    Batch procesing generator, yields a dict of sentences, candidates and labels if in training mode (determined if gold_file_path is specified)
    
    param batch_size: 
    param training_file_path: 
    param antivocab: 
    param output_vocab: 
    param gold_file_path: 
    return: generator object
    """
    batch = {"sentences" : [], "candidates" : []}
    
    training_data_flow = parsers.TrainingParser(training_file_path)
    if gold_file_path:
        gold_data_flow = parsers.GoldParser(gold_file_path)
        batch.update({"labels" : []})
        
    
    for batch_count, sentence in enumerate(training_data_flow.parse()):
        #training mode
        if gold_file_path:
            labels = gold_data_flow.parse()         
            output = prepare_sentence(sentence, antivocab, output_vocab, labels)
            
            batch['sentences'].append(output['sentence'])
            batch['candidates'].append(output['candidates'])
            batch['labels'].append(output['labels'])
        
        #evaulation mode
        else:
            output = prepare_sentence(sentence, antivocab, output_vocab)
            
            batch['sentences'].append(output['sentence'])
            batch['candidates'].append(output['candidates'])
            
        if batch_count == batch_size-1:

            for key in batch.keys():
                batch[key] = apply_padding(batch, key, maxlen = 50, value = 1)
            batch_count = 0
            yield (batch['sentences'][0], batch['labels'][0])
    del batch
    
            
        
def prepare_sentence(sentence, antivocab, output_vocab, labels=None):
    """
    Prepares an output sentence consisting of the sentence itself along with labels and candidates
    
    param sentence: 
    param antivocab: 
    param output_vocab: 
    param labels: 
    
    return output: dict with keys: sentence, labels, candidates all list type objects
    """
    records = namedtuple("Training", "id_ lemma pos instance")

    output = {"sentence" : [], "labels" : [], "candidates": []}
    for entry in sentence:
        
        id_, lemma, pos, _ = entry
        
        output_word = utils.replacement_routine(lemma, entry, antivocab, output_vocab)        
        output['sentence'].append(output_word)
        
        if id_ is None:
            output['labels'].append(output_word)
            candidates = [output_word]
            
        else:
            if labels is not None:
                current_label = labels.__next__()
                assert current_label.id_ == id_, "ID mismatch"
                
                sense = current_label.senses[0]
                sense = output_vocab[sense] if sense in output_vocab else output_vocab["<UNK>"]
                output['labels'].append(sense)
            candidates = utils.candidate_synsets(lemma, pos)
            candidates = [utils.replacement_routine(c, records(id_=None, lemma=c, pos="X", instance=True), antivocab, output_vocab) for c in candidates]
            
        output['candidates'].append(candidates)
    return output

batch = prepare_sentence_batch(batch_size = 1,
                               training_file_path = '../resources/WSD_Evaluation_Framework/Training_Corpora/SemCor/semcor.data.xml',
                               gold_file_path =  '../resources/WSD_Evaluation_Framework/Training_Corpora/SemCor/semcor.gold.key.txt',
                               antivocab = antivocab,
                               output_vocab = output_vocab)

In [7]:
#b['candidates'][0] + [[1]]*len(b['candidates'][0])

In [8]:
for b in batch:
    break

In [9]:
import models
keras.backend.clear_session()

In [10]:
NN = models.Basic()


In [11]:
model = NN.build(vocab_size = 47870,
                embedding_size = 32,
                hidden_size = 32,
                PADDING_SIZE = 50,
                LEARNING_RATE = 0.001,
                INPUT_DROPOUT = 0.2,
                LSTM_DROPOUT = 0.45,
                RECURRENT_DROPOUT = 0.35,
                N_EPOCHS = 10)

Creating KERAS model
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 50)                0         
_________________________________________________________________
embedding (Embedding)        (None, 50, 32)            1531840   
_________________________________________________________________
Bi-directional_LSTM (Bidirec (None, 50, 64)            16640     
_________________________________________________________________
time_distributed (TimeDistri (None, 50, 47870)         3111550   
Total params: 4,660,030
Trainable params: 4,660,030
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.fit_generator(batch, 
                    steps_per_epoch=2,
                    epochs=1, 
                    verbose=3,
                    callbacks=None,
                    validation_data=None,
                    validation_steps=None,
                    class_weight=None,
                    max_queue_size=10,
                    workers=-1, 
                    use_multiprocessing=False,
                    shuffle=False,
                    initial_epoch=0)

KeyboardInterrupt: 

In [18]:
!ls ../resources/WSD_Evaluation_Framework/Evaluation_Datasets/semeval2013

gold_BN.txt              gold_WNdomain.txt        semeval2013.data.xml
gold_LEX.txt             semeval2013              semeval2013.gold.key.txt
gold_WND.txt             semeval2013.csv          semeval2013csv
